In [ ]:
#IMPORTS
from google.colab import drive  #dar acceso a mi drive
import os                       #funciones del sistema operativo
import cv2                      #libreria para procesamiento de imagenes
from google.colab.patches import cv2_imshow #metodo de open cv para monstar imagenes, lo usaremos para verificar informacion
import numpy as np              #libreria para trabjar con arreglos y estructuras con muchas dimenciones
import matplotlib.pyplot as plt #libreria para hacer plots de los datos 

import pandas as pd

#sklearn => una de las mejores librerias de machine learning en python
from sklearn.metrics import confusion_matrix      #nos permite evaluar el desempeño del algoritmo, nos muestra las clases y cantidad con la cantidad predicha por el modelo, nos muestra si el algoritmos confunde clases
from sklearn.metrics import classification_report #nos permite ver la calidad de las predicciones realizadas por el modelo

import seaborn as sns           #libreria de visualizacion de datos

#imports para el modelado
from tensorflow.keras.layers import Dropout                     #"marginacion" capa que ayuda a evitar el overfiting, aleatoriamente escoje valores y los cambia a 0 en intervalos
from tensorflow.keras.layers import Flatten,BatchNormalization  #Aplanar(flatten) es la función que convierte el mapa de características agrupado en una sola columna que se pasa a la capa completamente conectada.
                                                                #La normalización por lotes(BatchNormalization) es una técnica diseñada para estandarizar automáticamente las entradas a una capa en una red neuronal de aprendizaje profundo.
from tensorflow.keras.layers import Dense, MaxPooling2D,Conv2D  #es una capa de convolución 2D, esta capa crea un núcleo de convolución que es entrada de capas que ayuda a producir un tensor de salida
from tensorflow.keras.layers import Input,Activation,Add
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split


In [ ]:
#Importo mi unidad de drive dando acceso para obtener los dataset y guardar contenido
drive.mount('/content/drive')

In [ ]:
#direccion donde esta mi proyecto
fldr = "drive/MyDrive/Colab Notebooks/Proyecto IA/"
files=  os.listdir(fldr+'/DataClean')

In [ ]:
#NO CORRER (solo si hay nuevo dataset), LA INFORMACION YA FUE TRANSFORMADA--------------------------------------------------------------------
#Dura casi 2 horas tratando los datos en colab
#Creamos los arreglos que contendran la informacion tomada del data set, cada uno se correspondera por los indices
ages    = []
genders = []
images  = []

#Estructura de los datos, cada imagen viene etiquetada con su respectiva edad y genero
#28_1_0_20170117122007867.jpg.chip.jpg

#Extraemos la informacion del dataset para prepararla para el modelo
for photo in files:
  age=int(photo.split('_')[0])
  gender=int(photo.split('_')[1])
  img_dir=fldr+'/DataClean/'+photo #url de la imagen
 
  image=cv2.imread(img_dir)

  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #simplificamos los tonos de color de las fotos para realzar la imagen
  image= cv2.resize(image,(48,48))               #estandarizamos el tamanno de la imagen pasando todas las imagenes en un formato de 48X48 pixceles esto generara un total de 2304 entradas para el modelo
  images.append(image)
  ages.append(age)
  genders.append(gender)
  #NO CORRER, LA INFORMACION YA FUE TRANSFORMADA--------------------------------------------------------------------

In [ ]:
#Comprobamos que los tamannos de los arreglos obtenidos del data set se correspondan mutuamente
print("TODOS deben ser del mismo largo: "," Edades " ,len(ages), " Generos ",len(genders), " Fotos "  ,len(images))

In [ ]:
#Elegimos una imagen aleatoria para corroborar que la imagen corresponda al genero y edad correspondiente aproximada
cv2_imshow(images[24])
print("Edad: ", ages[24], " Genero: ", genders[24])

In [ ]:
#Se transforman en arrays de pandas para guardar los datasets
images_=np.array(images)
genders_=np.array(genders)
ages_=np.array(ages)

In [ ]:
#guardo la infromacion para no tener tratarla nuevamente en futuras ejecuciones
#np.save(fldr+'/SalidasProyectoIA/image.npy',images_)
#np.save(fldr+'/SalidasProyectoIA/gender.npy',genders_)
#np.save(fldr+'/SalidasProyectoIA/age.npy',ages_)

In [ ]:
#NO USAR SI NO ES NECESARIO
#CON ESTA FUNCION CARGO LA DATA YA MANIPULADA-----------------------------------------------------
images_ = np.load("drive/MyDrive/Colab Notebooks/Proyecto IA/SalidasProyectoIA/image.npy")
genders_ = np.load("drive/MyDrive/Colab Notebooks/Proyecto IA/SalidasProyectoIA/gender.npy")
ages_  = np.load("drive/MyDrive/Colab Notebooks/Proyecto IA/SalidasProyectoIA/age.npy")
#CON ESTA FUNCION CARGO LA DATA YA MANIPULADA-----------------------------------------------------

In [ ]:
#Comprobamos que todos siguen teniendo el mismo tamanno
print("TODOS deben ser del mismo largo: "," Edades " ,len(ages_), " Generos ",len(genders_), " Fotos "  ,len(images_))

In [ ]:
#Me aseguro que solo existan dos generos 
values, counts = np.unique(genders_, return_counts=True)
print(counts ," ", values )

In [ ]:
#realizo un plot de los generos para ver la distribucion del data set
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
gender = ['Male', 'Female']
values=[counts[0],counts[1]] 
ax.bar(gender,values)
plt.xlabel('Genero')
plt.ylabel('Cantidad')
#fig.savefig(fldr+'/SalidasProyectoIA/images/generosPreEliminacion.png')
plt.show()


In [ ]:
#Visualizo las edades que estan presentes en la data
values, counts = np.unique(ages_, return_counts=True)
print(counts)

In [ ]:
#Vemos claramente una distribucion mal realizada en las edades
plt.hist(ages_, 50, facecolor='green', alpha=0.75)
plt.xlabel('Edad')
plt.ylabel('Cantidad')
#plt.savefig(fldr+'/SalidasProyectoIA/images/edadesPreEliminacion.png')

In [ ]:
'''#NO USAR, PERJUDICA EL MODELO-------------------------------------------------------------
#Se procede con la eliminacion de la informacion de los menores a 1 año

index_array = np.where(ages_ <= 4)[0] #creamos un nuevo array solo con la los indices de los elementos de edad de 1 año
#print(new_array) #indices de donde estan los elementos asociados a 1 anno


ages_2    = np.delete(ages_, index_array, axis=0) 
genders_2 = np.delete(genders_f, index_array, axis=0) 
images_2  = np.delete( images_, index_array,  axis=0)  
#NO USAR, PERJUDICA EL MODELO-------------------------------------------------------------'''

In [ ]:
cv2_imshow(images_[2999])
print(genders_[2999] ,  " ", ages_[2999])

In [ ]:
#index_array = np.where(ages_ <= 4)[0]
#min(index_array)

In [ ]:
#Comprobamos que todos siguen teniendo el mismo tamanno
print("TODOS deben ser del mismo largo: "," Edades " ,len(ages_2), " Generos ",len(genders_2), " Fotos "  ,len(images_2))


In [ ]:
cv2_imshow(images_2[2999])
print(genders_2[2999] ,  " ", ages_2[2999])

In [ ]:
#NUEVA DISTRIBUCION DE EDADES
#Aqui podemos observar una mejor distribucion, adicionalmente eliminamos los individuos de 1 año, los cuales presentan 
#caracteristicas androgenas y afectarian igualmente la prediccion de genero
plt.hist(ages_2, 50, facecolor='green', alpha=0.75)
plt.xlabel('Edad')
plt.ylabel('Cantidad')
#plt.savefig(fldr+'/SalidasProyectoIA/images/edadesPostEliminacion.png') #guardo una imagen del plot

In [ ]:
#NUEVA DISTRIBUCION DE GENEROS
#Observamos la nuevas distribucion de los generos, vemos que continuan muy similares

values, counts = np.unique(genders_2, return_counts=True)

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
gender = ['Male', 'Female']
values=[counts[0],counts[1]] 
ax.bar(gender,values)
plt.xlabel('Genero')
plt.ylabel('Cantidad')
#fig.savefig(fldr+'/SalidasProyectoIA/images/generosPostEliminacion.png')
plt.show()

In [ ]:
#Reemplazamos el valor de las variables anteriores
images_  = images_2
genders_ = genders_2
ages_    = ages_2

In [ ]:
ages_

In [ ]:
#Comprobamos que todos siguen teniendo el mismo tamanno
print("TODOS deben ser del mismo largo: "," Edades " ,len(ages_), " Generos ",len(genders_), " Fotos "  ,len(images_))

In [ ]:
#frecuencia de personas que tienen determinada edad
values, counts = np.unique(ages_, return_counts=True)
print(counts)

In [ ]:
#Otra manera de visualizar edades
val=values.tolist()
cnt=counts.tolist()
plt.plot(counts)
plt.xlabel('ages')
plt.ylabel('distribution')
plt.show()

In [ ]:
#Creamos las etiquetas que usaremos mas adelante para el modelo
labels=[]

i=0
#uniremos las edades y generos en un solo array como etiquetas
while i<len(ages_):
  label=[]
  label.append([ages_[i]])
  label.append([genders_[i]])
  labels.append(label)
  i+=1
#labels

In [ ]:
images_2=images_/255   #estandarizo el valor del color de los pixeles cuyo maximo valor es 255, aqui lo pongo entre un 1 y 0 cosa que debeficia al modelado
labels_f=np.array(labels) #le doy forma a las etiquetas de array de numpy
images_2.shape

In [ ]:
images_2[1] #Podemos apreciar los pixeles estandarizados, ahora ocupan valores entre 0 y 1

In [ ]:
labels_f=np.array(labels)
labels_f.shape

In [ ]:
#Separamos los datas entre entrenamiento y testeo con la siguiente funcion, usando 75% entrenamiento, 25% testeo
X_train, X_test, Y_train, Y_test= train_test_split(images_2, labels_f,test_size=0.25)


In [ ]:
#vemos las estiquetas de entrenamiento
Y_train[0:5]

In [ ]:
#vemos la data de entrenamiento
X_train[0:5]

In [ ]:
#Creamos un arreglo para entrenamiento y testeo combinando las dos features ( : significa tomar todo)
Y_train_2=[Y_train[:,1],Y_train[:,0]]
Y_test_2=[Y_test[:,1],Y_test[:,0]]

In [ ]:
Y_train_2[0][0:3]

In [ ]:
Y_train_2[1][0:3]

In [ ]:
#MODELO

def Convolution(input_tensor,filters):
    
    x = Conv2D(filters=filters,kernel_size=(3, 3),padding = 'same',strides=(1, 1),kernel_regularizer=l2(0.001))(input_tensor)
    x = Dropout(0.1)(x)
    x= Activation('relu')(x)

    return x
def model(input_shape):
  inputs = Input((input_shape))
  
  conv_1= Convolution(inputs,32)
  maxp_1 = MaxPooling2D(pool_size = (2,2)) (conv_1)
  conv_2 = Convolution(maxp_1,64)
  maxp_2 = MaxPooling2D(pool_size = (2, 2)) (conv_2)
  conv_3 = Convolution(maxp_2,128)
  maxp_3 = MaxPooling2D(pool_size = (2, 2)) (conv_3)
  conv_4 = Convolution(maxp_3,256)
  maxp_4 = MaxPooling2D(pool_size = (2, 2)) (conv_4)
  flatten= Flatten() (maxp_4)
  dense_1= Dense(64,activation='relu')(flatten)
  dense_2= Dense(64,activation='relu')(flatten)
  drop_1=Dropout(0.2)(dense_1)
  drop_2=Dropout(0.2)(dense_2)
  output_1= Dense(1,activation="sigmoid",name='sex_out')(drop_1)
  output_2= Dense(1,activation="relu",name='age_out')(drop_2)
  model = Model(inputs=[inputs], outputs=[output_1,output_2])
  model.compile(loss=["binary_crossentropy","mae"], optimizer="Adam",
	metrics=["accuracy"])
  
  return model

In [ ]:
Model=model((48,48,3))


In [ ]:
Model.summary()

In [ ]:

fle_s='sex_age_model.h5'
checkpointer = ModelCheckpoint(fle_s, monitor='val_loss',verbose=1,save_best_only=True,save_weights_only=False, mode='auto',save_freq='epoch')
Early_stop=tf.keras.callbacks.EarlyStopping(patience=75, monitor='val_loss',restore_best_weights=True),
callback_list=[checkpointer,Early_stop]

In [ ]:
History=Model.fit(X_train,Y_train_2,batch_size=64,validation_data=(X_test,Y_test_2),epochs=500,callbacks=[callback_list])
#Pare el modelo anticipadamente

In [ ]:
Model.evaluate(X_test,Y_test_2)

In [ ]:
# convierto el historico en un dataframe de pandas   
hist_df = pd.DataFrame(History.history) 

In [ ]:
#convierto el historico en csv y lo guardo
hist_df.to_csv(fldr+'/modelosGenerados/modelo2')


In [ ]:
#Cargo el historico guardado como csv
history = pd.read_csv(fldr+'/SalidasProyectoIA/history')

In [ ]:
pred=Model.predict(X_test)

In [ ]:
pred[1]

In [ ]:
#hago un plot para ver el comportamiento del modelo
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)

In [ ]:
plt.plot(History.history['sex_out_accuracy'])
plt.plot(History.history['age_out_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(Y_test_2[1], pred[1])
ax.plot([Y_test_2[1].min(),Y_test_2[1].max()], [Y_test_2[1].min(), Y_test_2[1].max()], 'k--', lw=4)
ax.set_xlabel('Actual Age')
ax.set_ylabel('Predicted Age')
plt.show()

In [ ]:
#guardo el modelo
# Guardar el Modelo
Model.save('drive/MyDrive/Colab Notebooks/Proyecto IA/modelosGenerados/Model2.h5')
# Recrea exactamente el mismo modelo solo desde el archivo
#new_model = keras.models.load_model('path_to_my_model.h5')


In [ ]:
i=0
Pred_l=[]
while(i<len(pred[0])):

  Pred_l.append(int(np.round(pred[0][i])))
  i+=1

In [ ]:
report=classification_report(Y_test_2[0], Pred_l)
print(report)

In [ ]:
results = confusion_matrix(Y_test_2[0], Pred_l)

In [ ]:
sns.heatmap(results, annot=True)

In [ ]:
def test_image(ind,images_, images_f_2, Model):
    image_test = images_f_2[ind]
    pred_1 = Model.predict(np.array([image_test]))
    sex_f = ['Male', 'Female']
    age = int(np.round(pred_1[1][0]))
    sex = int(np.round(pred_1[0][0]))
    print("Predicted Age: " + str(age))
    print("Predicted Sex: " + sex_f[sex])
    cv2_imshow( images_[ind])

In [ ]:
test_image(57,images_,images_2,Model)

print(ages_[57], " " , genders_[57] )

In [ ]:
test_image(137,images_,images_2,Model)

print(ages_[137], " " , genders_[137] )

In [ ]:
test_image(502,images_,images_2,Model)

print(ages_[502], " " , genders_[502] )

In [ ]:
#Model real test con elementos seleccionados aleatoriamente


import os
fldr = "drive/MyDrive/Colab Notebooks/Proyecto IA/testRandom"
filesTest=os.listdir(fldr)

images_original_size = []
imagesTest_f=[]

#Extraemos la informacion del dataset
for photo in filesTest:
  img_dir=fldr+'/'+photo #url de la imagen
 
  image=cv2.imread(img_dir)

  images_original_size.append(image)#para ver el tamanno original

  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #simplificamos los tonos de colos
  image= cv2.resize(image,(48,48))               #estandarizamos el tamanno de la imagen
  imagesTest_f.append(image)

temp_imges_test=np.array(imagesTest_f)
imagesTest_f_2=temp_imges_test/255 

In [ ]:
valorTest = 6
cv2_imshow(images_original_size[valorTest])
test_image(valorTest,imagesTest_f,imagesTest_f_2,Model)

In [ ]:
valorTest = 1
cv2_imshow(images_original_size[valorTest])
test_image(valorTest,imagesTest_f,imagesTest_f_2,Model)

In [ ]:
valorTest = 0
cv2_imshow(images_original_size[valorTest])
test_image(valorTest,imagesTest_f,imagesTest_f_2,Model)

In [ ]:
valorTest = 3
cv2_imshow(images_original_size[valorTest])
test_image(valorTest,imagesTest_f,imagesTest_f_2,Model)

In [ ]:
valorTest = 4
cv2_imshow(images_original_size[valorTest])
test_image(valorTest,imagesTest_f,imagesTest_f_2,Model)

In [ ]:
valorTest = 7
cv2_imshow(images_original_size[valorTest])
test_image(valorTest,imagesTest_f,imagesTest_f_2,Model)

In [ ]:
valorTest = 8
cv2_imshow(images_original_size[valorTest])
test_image(valorTest,imagesTest_f,imagesTest_f_2,Model)

In [ ]:
#cargo el modelo entrenado
from keras.models import load_model
model = load_model('drive/MyDrive/Colab Notebooks/Proyecto IA/modelosGenerados/Model.h5')


In [ ]:
img = tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=False, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
)
img

Convertimos el modelos a tensorflow js







In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install tensorflowjs

In [ ]:
'./drive/MyDrive/Colab Notebooks/'
!dir

In [ ]:
/content/

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/Proyecto IA/modeloFinal

In [ ]:
'./Model2.h5'

In [ ]:

!tensorflowjs_converter --input_format=keras ./Model2.h5 ./Model2

In [ ]:
const tf = require('@tensorflow/tfjs-node');

const ResNet50URI = `file:///${__dirname}/ResNet50/model.json`
const ResNet50 = await tf.loadLayersModel(ResNet50URI);